In [23]:
!kubectl get -n kafka-main-cluster kafkaconnectors debezium-connector-postgres -o yaml | yq

apiVersion: kafka.strimzi.io/v1beta2
kind: KafkaConnector
metadata:
  annotations:
    kubectl.kubernetes.io/last-applied-configuration: |
      {"apiVersion":"kafka.strimzi.io/v1beta2","kind":"KafkaConnector","metadata":{"annotations":{},"labels":{"strimzi.io/cluster":"my-connect"},"name":"debezium-connector-postgres","namespace":"kafka-main-cluster"},"spec":{"class":"io.debezium.connector.postgresql.PostgresConnector","config":{"converter.type":"avro","database.dbname":"airflow","database.hostname":"postgres-postgresql.postgres.svc","database.password":"${secrets:kafka-main-cluster/postgres-user-debezium:password}","database.port":5432,"database.user":"${secrets:kafka-main-cluster/postgres-user-debezium:username}","key.converter":"io.confluent.connect.avro.AvroConverter","key.converter.schema.registry.url":"http://main-registry-schema-registry.kafka-main-cluster.svc:8081","plugin.name":"decoderbufs","schema.history.internal.kafka.bootstrap.servers":"main-kafka-bootstrap:9092","schema

# Using Kubernetes Configuration Provider to load data from Secrets and Config Maps
   * https://strimzi.io/blog/2021/07/22/using-kubernetes-config-provider-to-load-data-from-secrets-and-config-maps/

In [23]:
!kubectl -n kafka-main-cluster get role connector-configuration-role -o yaml | yq

apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  annotations:
    kubectl.kubernetes.io/last-applied-configuration: |
      {"apiVersion":"rbac.authorization.k8s.io/v1","kind":"Role","metadata":{"annotations":{},"name":"connector-configuration-role","namespace":"kafka-main-cluster"},"rules":[{"apiGroups":[""],"resourceNames":["postgres-user-debezium"],"resources":["secrets"],"verbs":["get"]}]}
  creationTimestamp: "2024-01-31T21:11:15Z"
  name: connector-configuration-role
  namespace: kafka-main-cluster
  resourceVersion: "606513"
  uid: 11c59303-4a2d-4dfc-89bc-7dad23ed4627
rules:
  - apiGroups:
      - ""
    resourceNames:
      - postgres-user-debezium
    resources:
      - secrets
    verbs:
      - get


In [10]:
!kubectl -n kafka-main-cluster get rolebinding connector-configuration-role-binding -o yaml | yq

apiVersion: rbac.authorization.k8s.io/v1
kind: RoleBinding
metadata:
  annotations:
    kubectl.kubernetes.io/last-applied-configuration: |
      {"apiVersion":"rbac.authorization.k8s.io/v1","kind":"RoleBinding","metadata":{"annotations":{},"name":"connector-configuration-role-binding","namespace":"kafka-main-cluster"},"roleRef":{"apiGroup":"rbac.authorization.k8s.io","kind":"Role","name":"connector-configuration-role"},"subjects":[{"kind":"ServiceAccount","name":"my-connect-connect","namespace":"kafka-main-cluster"}]}
  creationTimestamp: "2024-01-31T21:11:15Z"
  name: connector-configuration-role-binding
  namespace: kafka-main-cluster
  resourceVersion: "606514"
  uid: 71900323-e3db-4879-bdf2-c772b891dcf4
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: Role
  name: connector-configuration-role
subjects:
  - kind: ServiceAccount
    name: my-connect-connect
    namespace: kafka-main-cluster


In [14]:
!kubectl -n kafka-main-cluster get secrets postgres-user-debezium -o yaml | yq

apiVersion: v1
data:
  password: ZGViZXppdW0K
  username: ZGViZXppdW0K
kind: Secret
metadata:
  annotations:
    replicator.v1.mittwald.de/replicated-at: "2024-01-31T16:10:45Z"
    replicator.v1.mittwald.de/replicated-from-version: "19115"
    replicator.v1.mittwald.de/replicated-keys: password,username
  creationTimestamp: "2024-01-31T16:10:45Z"
  name: postgres-user-debezium
  namespace: kafka-main-cluster
  resourceVersion: "19116"
  uid: 8434a02b-5d8e-42f8-bd19-6947356c3e03
type: Opaque


In [4]:
!curl -s -H "Accept:application/json" http://my-connect-connect-api.kafka-main-cluster.svc:8083/connector-plugins/ | jq

[
  {
    "class": "io.debezium.connector.mongodb.MongoDbConnector",
    "type": "source",
    "version": "2.5.0.Final"
  },
  {
    "class": "io.debezium.connector.mysql.MySqlConnector",
    "type": "source",
    "version": "2.5.0.Final"
  },
  {
    "class": "io.debezium.connector.oracle.OracleConnector",
    "type": "source",
    "version": "2.5.0.Final"
  },
  {
    "class": "io.debezium.connector.postgresql.PostgresConnector",
    "type": "source",
    "version": "2.5.0.Final"
  },
  {
    "class": "io.debezium.connector.sqlserver.SqlServerConnector",
    "type": "source",
    "version": "2.5.0.Final"
  },
  {
    "class": "org.apache.kafka.connect.mirror.MirrorCheckpointConnector",
    "type": "source",
    "version": "3.6.1"
  },
  {
    "class": "org.apache.kafka.connect.mirror.MirrorHeartbeatConnector",
    "type": "source",
    "version": "3.6.1"
  },
  {
    "class": "org.apache.kafka.connect.mirror.MirrorSourceConnector",
    "type": "source",
    "version": "3.6.1"
  }
]


In [11]:
!curl -s -H "Accept:application/json" http://my-connect-connect-api.kafka-main-cluster.svc:8083/connector-plugins/source/config/validate | jq

{
  "error_code": 405,
  "message": "HTTP 405 Method Not Allowed"
}


In [12]:
!kubectl -n kafka get serviceaccount strimzi-cluster-operator -o yaml | yq

apiVersion: v1
kind: ServiceAccount
metadata:
  annotations:
    meta.helm.sh/release-name: strimzi-cluster-operator
    meta.helm.sh/release-namespace: kafka
  creationTimestamp: "2024-02-01T13:17:22Z"
  labels:
    app: strimzi
    app.kubernetes.io/managed-by: Helm
    chart: strimzi-kafka-operator-0.39.0
    component: service-account
    heritage: Helm
    release: strimzi-cluster-operator
  name: strimzi-cluster-operator
  namespace: kafka
  resourceVersion: "10547"
  uid: 5e9f1db2-cafc-4324-9ba9-22b7621f18ce


In [9]:
!kubectl -n kafka get roles -o yaml | yq

apiVersion: v1
items: []
kind: List
metadata:
  resourceVersion: ""


In [4]:
!kubectl -n kafka get rolebinding  strimzi-cluster-operator -o yaml | yq

apiVersion: rbac.authorization.k8s.io/v1
kind: RoleBinding
metadata:
  annotations:
    meta.helm.sh/release-name: strimzi-cluster-operator
    meta.helm.sh/release-namespace: kafka
  creationTimestamp: "2024-02-01T13:17:22Z"
  labels:
    app: strimzi
    app.kubernetes.io/managed-by: Helm
    chart: strimzi-kafka-operator-0.39.0
    component: role-binding
    heritage: Helm
    release: strimzi-cluster-operator
  name: strimzi-cluster-operator
  namespace: kafka
  resourceVersion: "10575"
  uid: 2d2c3f40-8811-439b-a916-2bd7665cf975
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: strimzi-cluster-operator-namespaced
subjects:
  - kind: ServiceAccount
    name: strimzi-cluster-operator
    namespace: kafka
